In [1]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.version.cuda)
print('Location:', torch.__file__) # /home/emre/.local/lib/python3.7/site-packages

1.12.0+cu102
True
10.2
Location: /home/emre/anaconda3/envs/emre_venv/lib/python3.9/site-packages/torch/__init__.py


/home/emre/anaconda3/envs/emre_venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from typing import Union, Tuple
from torch_geometric.typing import OptPairTensor, Adj, Size # Optional[Tensor], Union[Tensor, SparseTensor], Optional[Tuple[int, int]], all about data type

from torch import Tensor
from torch.nn import Linear
import torch.nn.functional as F
from torch_sparse import SparseTensor, matmul
from torch_geometric.nn.conv import MessagePassing

In [3]:
import warnings
warnings.filterwarnings("ignore")
from joblib import Parallel, delayed

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.nn as gnn
from torchsummary import summary
import torchvision
from torch_geometric.nn import MessagePassing
from torch_geometric.data import Dataset, Data, DataLoader
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from torch_geometric.utils import softmax

In [4]:
import pickle
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
!CUDA_LAUNCH_BLOCKING=1
torch.cuda.set_device(1)
print('Num GPU:', torch.cuda.device_count())
print('Current device:', torch.cuda.current_device())
print('Device name:', torch.cuda.get_device_name())

Num GPU: 3
Current device: 1
Device name: TITAN V


In [5]:
def load_pkl(filename):
    with open(filename, 'rb') as input:
        data = pickle.load(input)
    return data

In [24]:
[train_images, train_labels,
 test_images, test_labels] = load_pkl('datasets/WLASL_100_mDsim.pkl')
train_images = np.swapaxes(train_images, 1, 3)
test_images = np.swapaxes(test_images, 1, 3)
train_labels = np.argmax(train_labels, -1)
test_labels = np.argmax(test_labels, -1)

print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

(1567, 3, 224, 224)
(1567,)
(393, 3, 224, 224)
(393,)


In [65]:
num_class = 10
train_idx = [i for i in range(len(train_labels)) if train_labels[i] < num_class]
test_idx = [i for i in range(len(test_labels)) if test_labels[i] < num_class]
train_images = np.array([train_images[idx] for idx in train_idx])
train_labels = np.array([train_labels[idx] for idx in train_idx])
test_images = np.array([test_images[idx] for idx in test_idx])
test_labels = np.array([test_labels[idx] for idx in test_idx])

print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

(226, 3, 224, 224)
(226,)
(54, 3, 224, 224)
(54,)


## Dataloader

In [66]:
class CustomImageDataset(Dataset):
    def __init__(self, x, y):
        super().__init__()
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        temp_x = torch.tensor(self.x[idx], dtype=torch.float)
        temp_y = torch.tensor(self.y[idx], dtype=torch.long)
#         self.x = self.x[idx]
#         self.y = self.y[idx]
#         return self.x[idx], self.y[idx] 
        return temp_x, temp_y

## Hyperparams

In [67]:
batch_size = 32

## Classify mD

In [68]:
training_data = CustomImageDataset(torch.from_numpy(train_images), torch.from_numpy(train_labels))
train_loader = DataLoader(training_data, batch_size=batch_size, shuffle=False, collate_fn=lambda x: x)
testing_data = CustomImageDataset(torch.from_numpy(test_images), torch.from_numpy(test_labels))
test_loader = DataLoader(testing_data, batch_size=batch_size, shuffle=False, collate_fn=lambda x: x)

In [69]:
training_features, training_labels = next(iter(train_loader))
print(f"Feature batch shape: {training_features.size()}")
print(f"Labels batch shape: {training_labels.size()}")

Feature batch shape: torch.Size([32, 3, 224, 224])
Labels batch shape: torch.Size([32])


In [70]:
# # iterate through the dataset:
# for i, batch in enumerate(test_loader):
#     print(f'{i}', '. batch length:', f'{len(batch[0])}')

In [71]:
%matplotlib inline
def train( model, model_name, prog_bar = True):
    
    max_test_acc = -1
    train_accs = []
    test_accs = []
    
    for epoch in range(epochs):
        train_acc = []
        test_acc = []
        train_loss = 0
        test_loss = 0
        if prog_bar:
            pbar = tqdm(train_loader,position=0)
        else:
            pbar = train_loader
        
        # train
        for data in pbar:
            x, labels = data
            x = x.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            out = model(x)
            loss = criterion(out, labels)
            train_loss += loss.cpu().detach().numpy()/len(train_loader)
            loss.backward()
            optimizer.step()
            preds = np.argmax(out.cpu().detach().numpy(), -1)
            labels = labels.cpu().detach().numpy()
#             print('preds', preds)
#             print('labels', labels)
            
            acc = np.sum(preds == labels) / len(labels) * 100
            train_acc.append(acc)
            if prog_bar:
                pbar.set_description('Train_acc: '+str(round(acc,2)))
        train_accs.append(np.mean(train_acc))
        
        # test
        all_preds = []
        for data in test_loader:
            x, labels = data
            x = x.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            out = model(x)
            loss = criterion(out, labels)
            test_loss += loss.cpu().detach().numpy()/len(test_loader)
            preds = np.argmax(out.cpu().detach().numpy(), -1)
            labels = labels.cpu().detach().numpy()
            acc = np.sum(preds == labels) / len(labels) * 100
            test_acc.append(acc)
            all_preds.append(preds)
        test_accs.append(np.mean(test_acc))
        
        if np.mean(test_acc) > max_test_acc:
#             print(np.array(all_preds[0]).shape)
#             print(np.array(all_preds[1]).shape)
            best_preds = [i for a in all_preds for i in a]
            max_test_acc = np.mean(test_acc)
            torch.save(model.state_dict(), 'models/' + model_name + str(round(max_test_acc, 2)) + '.pth')
        
#         lr_scheduler.step(test_loss)
        print('Epoch: ', str(epoch+1)+'/'+str(epochs),'| Training Acc: ', round(np.mean(train_acc), 2), '| Testing Acc: ', round(np.mean(test_acc), 2))
#         train_losses.append(train_loss)
#         test_losses.append(test_loss)

        
        lr_scheduler.step(train_loss)
        if not prog_bar:
            clear_output(wait=True)
            print('Epoch: ', str(epoch+1)+'/'+str(epochs),'| Training Acc: ', round(np.mean(train_acc), 2), '| Testing Acc: ', round(np.mean(test_acc), 2),
                 ', Best Testing Acc:', round(max_test_acc, 2))
            plt.plot(train_accs, label="Train Acc")
            plt.plot(test_accs, label="Validation Acc")
            plt.xlabel("# Epoch")
            plt.ylabel("Accuracy")
            plt.legend(loc='upper right')
            plt.show()
    return best_preds

In [72]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, 3)
        self.conv11 = nn.Conv2d(64, 64, 3)
        self.conv2 = nn.Conv2d(64, 128, 3)
        self.conv21 = nn.Conv2d(128, 128, 3)
        self.conv3 = nn.Conv2d(128, 128, 3)
        self.conv31 = nn.Conv2d(128, 128, 3)
#         self.conv4 = nn.Conv2d(128, 128, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(14*14*128, 128) # 3 layers: 14*14*128, 4 layers: 6*6*128
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 100)
        self.drop = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
#         x = F.relu(self.conv11(x))
        x = self.pool(F.relu(self.conv2(x)))
#         x = F.relu(self.conv21(x))
        x = self.pool(F.relu(self.conv3(x)))
#         x = F.relu(self.conv31(x))
#         x = self.pool(F.relu(self.conv4(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.drop(x)
        x = F.relu(self.fc1(x))
        x = self.drop(x)
        x = F.relu(self.fc2(x))
        x = self.drop(x)
        x = F.softmax(self.fc3(x))
        return x

In [32]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN().to(device)
summary(model, (3, 128, 128))
print('device:', device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 126, 126]           1,792
         MaxPool2d-2           [-1, 64, 63, 63]               0
            Conv2d-3          [-1, 128, 61, 61]          73,856
         MaxPool2d-4          [-1, 128, 30, 30]               0
            Conv2d-5          [-1, 128, 28, 28]         147,584
         MaxPool2d-6          [-1, 128, 14, 14]               0
           Dropout-7                [-1, 25088]               0
            Linear-8                  [-1, 128]       3,211,392
           Dropout-9                  [-1, 128]               0
           Linear-10                  [-1, 128]          16,512
          Dropout-11                  [-1, 128]               0
           Linear-12                  [-1, 100]          12,900
Total params: 3,464,036
Trainable params: 3,464,036
Non-trainable params: 0
---------------------------

In [33]:
model_name = 'model_cnn_mDsim_imit_acc'
epochs = 200
best_preds = train(model, model_name, prog_bar = False)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x86528 and 25088x128)

In [34]:
acc = np.sum([1 for i in range(len(test_labels)) if test_labels[i] == best_preds[i]]) / len(test_labels) * 100
acc

NameError: name 'best_preds' is not defined

# ResNet-50

In [73]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)
resnet = torchvision.models.resnet18(pretrained=True)
# resnet = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True).to(device)
resnet.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(
        in_features=512, # 2048 for resnet-50, 512 for 18
        out_features=num_class),
#     nn.ReLU(),
#     nn.Dropout(0.5),
#     nn.Linear(
#         in_features=256, # 2048 for resnet-50, 512 for 18
#         out_features=10),
    nn.Softmax())
resnet.eval().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet.parameters(), lr=0.001)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, patience=20, factor=0.5)

device: cuda


Epoch:  7/200 | Training Acc:  10.94 | Testing Acc:  13.07 , Best Testing Acc: 13.07


KeyboardInterrupt: 

In [74]:
model_name = 'model_resnet18_withTL_mDsim_acc'
# model_name = 'model_vgg19_withTL_mDsim_imit_acc' # 65.38
epochs = 200
best_preds = train(resnet, model_name, prog_bar = False)

In [ ]:
acc = np.sum([1 for i in range(len(test_labels)) if test_labels[i] == best_preds[i]]) / len(test_labels) * 100
acc

# squeezenet1_0

In [75]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)
resnet = torchvision.models.squeezenet1_0(pretrained=True)
# resnet = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True).to(device)
resnet.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(
        in_features=2048, # 2048 for resnet-50, 512 for 18
        out_features=num_class),
#     nn.ReLU(),
#     nn.Dropout(0.5),
#     nn.Linear(
#         in_features=64, # 2048 for resnet-50, 512 for 18
#         out_features=10),
    nn.Softmax())
resnet.eval().to(device)
# resnet.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet.parameters(), lr=0.001)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, patience=20, factor=0.5)

device: cuda


Epoch:  200/200 | Training Acc:  100.0 | Testing Acc:  37.64 , Best Testing Acc: 47.59


In [76]:
# model_name = 'model_resnet50_withTL_mDsim_imit_acc'
model_name = 'model_squeezenet1_0_withTL_mDsim_acc' # 78.85
epochs = 200
best_preds = train(resnet, model_name, prog_bar = False)

In [77]:
acc = np.sum([1 for i in range(len(test_labels)) if test_labels[i] == best_preds[i]]) / len(test_labels) * 100
acc

46.2962962962963

# googlenet

In [78]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)
resnet = torchvision.models.mnasnet1_0(pretrained=True)
# resnet = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True).to(device)
resnet.fc = nn.Sequential(
    nn.Linear(
        in_features=2048, # 2048 for resnet-50, 512 for 18
        out_features=num_class),
#     nn.ReLU(),
#     nn.Dropout(0.5),
#     nn.Linear(
#         in_features=256, # 2048 for resnet-50, 512 for 18
#         out_features=10),
    nn.Softmax())
resnet.eval().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet.parameters(), lr=0.001)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, patience=20, factor=0.5)

device: cuda


Epoch:  200/200 | Training Acc:  7.03 | Testing Acc:  5.4 , Best Testing Acc: 13.78


In [79]:
model_name = 'model_mnasnet1_0_withTL_mDsim_acc' 
epochs = 200
best_preds = train(resnet, model_name, prog_bar = False)

In [80]:
acc = np.sum([1 for i in range(len(test_labels)) if test_labels[i] == best_preds[i]]) / len(test_labels) * 100
acc

12.962962962962962

# VGG-16

In [81]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)
vgg16 = torchvision.models.vgg16(pretrained=True)
    
vgg16.fc = torch.nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(
        in_features=2048,
        out_features=32
    ),
    nn.ReLU(),    
    nn.Dropout(0.5),
    nn.Linear(
        in_features=32,
        out_features=num_class),
    nn.Softmax()
)

# for param in vgg16.parameters():
#     param.requires_grad = False

# for param in vgg16.fc.parameters():
#     param.requires_grad = True
# vgg16.fc.requires_grad_(True)
vgg16.to(device).eval()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vgg16.parameters(), lr=0.001)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, verbose=True, patience=20, factor=0.5)

device: cuda


Epoch:  130/500 | Training Acc:  96.48 | Testing Acc:  23.01 , Best Testing Acc: 38.35


KeyboardInterrupt: 

In [82]:
model_name = 'model_vgg16_withTL_mDsim_acc' # 67.07 best
epochs = 500
best_preds = train(vgg16, model_name, prog_bar = False)

In [ ]:
acc = np.sum([1 for i in range(len(test_labels)) if test_labels[i] == best_preds[i]]) / len(test_labels) * 100
acc